In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('resultado_unificado.csv')

In [8]:
df['Especie'].unique()

array(['ALMEJA', 'ATUN', 'BANDERA', 'BAQUETA', 'BARRILETE', 'BERRUGATA',
       'BONITO', 'CABRILLA', 'CALAMAR', 'CAMARON', 'CARACOL', 'CAZON',
       'CHARAL', 'CORVINA', 'ERIZO', 'ESMEDREGAL', 'GUACHINANGO', 'JAIBA',
       'JUREL', 'LANGOSTA', 'LEBRANCHA', 'LENGUADO', 'LISA', 'LOBINA',
       'MERO', 'MOJARRA', 'OSTION', 'OTRAS', 'PAMPANO', 'PARGO', 'PIERNA',
       'PULPO', 'RAYA Y SIMILARES', 'ROBALO', 'RONCO', 'SARDINA',
       'SARGAZO', 'SIERRA', 'TIBURON'], dtype=object)

In [9]:
from species_classifier import classify_species

# Define el diccionario de hábitats
habitat_groups = {
    'ALMEJA': 'Bentónicas',
    'CAMARON': 'Bentónicas',
    'JAIBA': 'Bentónicas',
    'CARACOL': 'Bentónicas',
    'LANGOSTA': 'Bentónicas',
    'PULPO': 'Bentónicas',
    'RAYA Y SIMILARES': 'Bentónicas',
    'ERIZO': 'Bentónicas',
    'OSTION': 'Bentónicas',
    'ATUN': 'Pelágicas',
    'BARRILETE': 'Pelágicas',
    'BONITO': 'Pelágicas',
    'JUREL': 'Pelágicas',
    'PAMPANO': 'Pelágicas',
    'SIERRA': 'Pelágicas',
    'SARDINA': 'Pelágicas',
    'CABRILLA': 'Arrecifes',
    'MERO': 'Arrecifes',
    'GUACHINANGO': 'Arrecifes',
    'TIBURON': 'Pelágicas',
    'BAQUETA': 'Arrecifes',
    'ESMEDREGAL': 'Arrecifes',
    'LEBRANCHA': 'Costeras',
    'LISA': 'Costeras',
    'MOJARRA': 'Costeras',
    'ROBALO': 'Costeras',
    'CORVINA': 'Costeras',
    'CHARAL': 'Costeras',
    'SARGAZO': 'Otras',
    'BANDERA': 'Costeras',
    'LENGUADO': 'Otras',
    'PARGO': 'Arrecifes',
    'PIERNA': 'Arrecifes',
    'RONCO': 'Arrecifes',
    'LOBINA': 'Otras',
    'OTRAS': 'Otras'
}

# Ruta al archivo con datos de especies
file_path = 'resultado_unificado.csv'

# Llamar a la función
classified_data = classify_species(file_path, habitat_groups, output_path='classified_species_output.csv')

ModuleNotFoundError: No module named 'species_classifier'